In [2]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [5]:
cd

/Users/keithwalcott


In [61]:
codon_variant = pd.read_csv('kw_dms/results/variants/codon_variants.csv', usecols=['barcode','library','barcode','codon_substitutions','aa_substitutions','n_codon_substitutions','n_aa_substitutions'])
sample_4u8c_B1T1 = pd.read_csv('kw_dms/old_results/barcode_counts/LibB-231017-4u8c_bio1-1_counts.csv')
sample_DMSO_B1T1 = pd.read_csv('kw_dms/old_results/barcode_counts/LibB-231017-DMSO_bio1-1_counts.csv')
BD_calc = pd.read_csv('kw_dms/old_results/func_scores/4u8c_B1T1-vs-B1T1_func_scores.csv')

In [110]:
def calculate_functional_score(pre_selection, post_selection, codon_variant, library, count_threshold):
    # Rename count columns
    pre_selection.rename(columns={'count': 'count_pre'}, inplace=True)
    post_selection.rename(columns={'count': 'count_post'}, inplace=True)
    
    # Filter out barcodes in pre_selection that dont meet the count and % threshold
    # total_pre_counts = pre_selection['count_pre'].sum()
#     pre_selection = pre_selection[(pre_selection['count_pre'] >= count_threshold) & 
#                                   (pre_selection['count_pre'] / total_pre_counts >= 0.0000001)]   
    
    pre_selection = pre_selection[(pre_selection['count_pre'] >= count_threshold)] 
    
    # Merge codon_variants with 
    pre_selection = pd.merge(pre_selection, codon_variant, on='barcode')
    post_selection = pd.merge(post_selection, codon_variant, on='barcode')
    
    #filter out barcodes not in intended library
    pre_selection = pre_selection[pre_selection['library'] == library]
    post_selection = post_selection[post_selection['library'] == library]
    
    # Merge pre and post counts
    merged_df = pd.merge(pre_selection, post_selection[['barcode', 'count_post']], on='barcode', how='left')
    merged_df.fillna(0, inplace=True)

    # Change zeroes in 'count_post' to 0.5
    # Convert 'count_post' column to float
    merged_df['count_post'] = merged_df['count_post'].astype(float)

    # Now change zeroes in 'count_post' to 0.5
    merged_df.loc[merged_df['count_post'] == 0, 'count_post'] = 0.5
    
    # Calculate total wt counts then filter them out of the df
    wt_barcodes = merged_df[merged_df['n_codon_substitutions'] == 0]
    pre_wt_counts = wt_barcodes['count_pre'].sum()
    post_wt_counts = wt_barcodes['count_post'].sum()
    merged_df = merged_df[merged_df['n_codon_substitutions'] > 0]
    
    # Calculate the functional score for each row
    merged_df['func_score'] = np.log2((merged_df['count_post'] / post_wt_counts) / (merged_df['count_pre'] / pre_wt_counts))
    merged_df = merged_df.sort_values(by='barcode', ascending=True)
    merged_df = merged_df[['barcode','func_score','n_aa_substitutions','n_codon_substitutions','count_pre','count_post']]
    
    return merged_df


In [112]:
my_calc = calculate_functional_score(sample_DMSO_B1T1,sample_4u8c_B1T1,codon_variant,'LibB',25)
my_calc

,func_score,n_aa_substitutions,n_codon_substitutions,count_pre,count_post
3333,-0.219980,1,2,1399,1080.0
77510,1.401312,4,5,32,76.0
20135,-1.026322,0,1,444,196.0
49888,1.038346,2,2,137,253.0
68713,-5.868983,2,2,65,1.0
...,...,...,...,...,...
46367,1.536867,4,5,156,407.0
54954,-7.641032,4,5,111,0.5
21944,0.261300,1,1,412,444.0
38875,-8.567715,1,1,211,0.5


In [108]:
BD_calc = BD_calc.sort_values(by='barcode', ascending=True)
BD_calc.describe()

,func_score,func_score_var,n_aa_substitutions,n_codon_substitutions
count,71418.000000,71418.000000,71418.000000,71418.000000
mean,-1.143344,0.335431,1.528326,1.729578
std,2.467328,0.943152,1.310003,1.412442
min,-9.982000,0.000282,0.000000,0.000000
25%,-1.686000,0.009910,1.000000,1.000000
50%,-0.347000,0.022600,1.000000,2.000000
75%,0.430675,0.052870,2.000000,3.000000
max,4.318000,4.197000,38.000000,41.000000
